In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colormaps

In [ ]:
TEST = False

In [ ]:
# Define length of rope
R=10

In [ ]:
if TEST==1:
    filename = "data/input_9_test"
elif TEST==2:
    filename = "data/input_9_test_2"
else:
    filename = "data/input_9"

In [ ]:
with open(filename) as file:
    input_str = file.read()

In [ ]:
instructions = [line.split(' ') for line in input_str.strip('\n').split('\n')]
instructions[:5]

In [ ]:
directions = {'R':(1,0),
              'L':(-1,0),
              'U':(0,1),
              'D':(0,-1)}

In [ ]:
moves = [(directions[move[0]],int(move[1])) for move in instructions]
moves[:5]

In [ ]:
class Position:
       
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.trail = [(x,y)]
    
    def move(self, x, y):
        self.x += x
        self.y += y
        self.trail += [(self.x,self.y)]
        
    def distance(self, position):
        x, y = position.x, position.y
        x_dif = self.x - x
        y_dif = self.y - y
        return Position(x_dif, y_dif)        
    
    def __str__(self):
        return "({0},{1})".format(self.x,self.y)
    
    def __repr__(self):
        return "({0},{1})".format(self.x,self.y)  

In [ ]:
def move_tail(H, T, verbose=False, i=''):
    '''Move the tail position to catch up with the head'''
    dist = H.distance(T)
    if verbose: print("  distance from knot {1} to knot {2} is {0}".format(dist,i+1,i))
    if verbose: print("  knot {2} moved from ({0},{1})".format(T.x,T.y,i+1), end=' ')
    if abs(dist.x)<=1 and abs(dist.y)<=1:
        pass
    elif abs(dist.x)>1 and dist.y==0:
        T.move(np.sign(dist.x), 0)
    elif dist.x==0 and abs(dist.y)>1:
        T.move(0,np.sign(dist.y))
    elif abs(dist.x)>1 and abs(dist.y)==1:
        T.move(np.sign(dist.x),np.sign(dist.y))
    elif abs(dist.x)==1 and abs(dist.y)>1:
        T.move(np.sign(dist.x),np.sign(dist.y))
    elif abs(dist.x)==2 and abs(dist.y)==2:
        T.move(np.sign(dist.x),np.sign(dist.y))
    else:
        raise ValueError("knot {1} too far away from knot {2} - at {0} distance!".format(dist,i+1,i))
    if verbose: print(" to ({0},{1})\n".format(T.x,T.y))


In [ ]:
def move_heads(rope,vector,n_steps,verbose=True):
    '''Move the rope's head and all following knots iteratively'''
    for i in range(n_steps):
        if verbose: print(" head moved from {0}".format(rope[0]), end=' ')
        rope[0].move(*vector)
        if verbose: print(" to {0}".format(rope[0]))
        for i in range(R-1):
            move_tail(rope[i],rope[i+1], verbose, i) 
    
    

In [ ]:
def execute_rope_moves(moves, verbose=False, plot=True):
    '''Move a rope with all it's knots according to the list of moves'''
    rope = [Position(0,0) for i in range(R)]
    if plot: render_rope(rope)
    for move in moves:
        if verbose: print("Head: {0}, Tail: {1}, moving {2} steps in {3} direction".format(rope[0],rope[-1],move[1],move[0]))
        move_heads(rope,move[0],move[1], verbose)
        if plot: render_rope(rope)
    return rope

Get the boundaries of the region where the rope will move based on the list of moves. This is used in render_rope().

In [ ]:
x = [0]
y = [0]
for instruction in instructions:
    direction, steps = instruction
    steps = int(steps)
    match direction:
        case 'R':
            x.append(x[-1] + steps)
        case 'L':
            x.append(x[-1] - steps)
        case 'U':
            y.append(y[-1] + steps)
        case 'D':
            y.append(y[-1] - steps)     

x_max = max(x)
x_min = min(x)
y_max = max(y)
y_min = min(y)
print("X range: {0} to {1}, Y range: {2} to {3}".format(x_min, x_max, y_min, y_max))

In [ ]:
def render_rope(rope):
    '''Plot current position of the rope with all it's knots on a grid'''
    grid = pd.DataFrame(-10,columns = list(range(x_min, x_max+1)), index = list(range(y_min, y_max+1)))
    for i in range(len(rope)):
        grid.loc[rope[i].y,rope[i].x] = i

    cmap = colormaps['plasma']
    fig, ax = plt.subplots(1,1,figsize=(2,2))
    ax.imshow(grid[::-1], cmap=cmap, interpolation='none', extent=[x_min,x_max+1,y_min,y_max+1])
      

In [ ]:
# Recommend verbose=False for long rope
# Recommend plot=False for non-test input
rope_end = execute_rope_moves(moves, verbose=False, plot=False)

In [ ]:
n_unique_tail_positions = len(set(rope_end[-1].trail))

In [ ]:
match TEST, R:
    case 1, 2:
        TEST_ANSWER=13
    case 1, 10:
        TEST_ANSWER=1
    case 2, 10:
        TEST_ANSWER=36
        

In [ ]:
if TEST:
    assert n_unique_tail_positions == TEST_ANSWER
else: 
    print("Tail appeared at {0} unique positions".format(n_unique_tail_positions))

Visualise all the positions where the tail has appeared

In [ ]:
tail_trail = [Position(*t) for t in rope_end[-1].trail]
render_rope(tail_trail)